
## Max Flow Optimization Example

In this tutorial, we want to solve the following examples:

<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_91.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_92.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_93.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_94.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_95.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_96.webp?raw=1\" width="700">

### Install OR-Tools

In [ ]:
pip install --upgrade --user ortools

### Declare the solver
In any MIP program, you start by importing the linear solver wrapper and declaring the MIP solver, as shown in the previous MIP example.

In [10]:
from ortools.linear_solver import pywraplp

### Create the data
The following code creates arrays containing the data for the example: the variable coefficients for the constraints and objective function, and bounds for the constraints.

In [24]:
Nodes = [i for i in range(1,7)]
Capacities = [[0,3,10,2,0,0],
              [1,0,0,1,0,2],
              [0,0,0,3,2,0],
              [0,1,1,0,0,1],
              [0,0,1,0,0,6],
              [1000,2,0,1,0,0]]


### Instantiate the solver
The following code instantiates the solver.

In [31]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

### Define the variables
The following code defines the variables for the example in a loop. For large problems, this is easier than defining the variables individually, as in the previous example.

In [32]:
infinity = solver.infinity()
x = {}
for i in Nodes:
    for j in Nodes:
        x[i,j] = solver.IntVar(0, infinity, f'x[{i},{j}]')

print('Number of variables =', solver.NumVariables())

Number of variables = 36


In [33]:
x

{(1, 1): x[1,1],
 (1, 2): x[1,2],
 (1, 3): x[1,3],
 (1, 4): x[1,4],
 (1, 5): x[1,5],
 (1, 6): x[1,6],
 (2, 1): x[2,1],
 (2, 2): x[2,2],
 (2, 3): x[2,3],
 (2, 4): x[2,4],
 (2, 5): x[2,5],
 (2, 6): x[2,6],
 (3, 1): x[3,1],
 (3, 2): x[3,2],
 (3, 3): x[3,3],
 (3, 4): x[3,4],
 (3, 5): x[3,5],
 (3, 6): x[3,6],
 (4, 1): x[4,1],
 (4, 2): x[4,2],
 (4, 3): x[4,3],
 (4, 4): x[4,4],
 (4, 5): x[4,5],
 (4, 6): x[4,6],
 (5, 1): x[5,1],
 (5, 2): x[5,2],
 (5, 3): x[5,3],
 (5, 4): x[5,4],
 (5, 5): x[5,5],
 (5, 6): x[5,6],
 (6, 1): x[6,1],
 (6, 2): x[6,2],
 (6, 3): x[6,3],
 (6, 4): x[6,4],
 (6, 5): x[6,5],
 (6, 6): x[6,6]}

### Define the constraints

The following code creates the constraints for the example, using loop.


In [34]:
# Capacity of each arc:
# You can also define this constraint as an upper bound of each decision variable in previous step
for i in Nodes:
    for j in Nodes:
        solver.Add(x[i,j] <= Capacities[i-1][j-1]) # this -1 is because indexing in python starts from 0. 
                                                   # E.g. the capacity of node 1 to node 2 is Capacities[0,1]

# flow balance at each node:
for i in Nodes:
    inflow = []
    outflow = []
    for j in Nodes:
        inflow.append(x[j,i])
        outflow.append(x[i,j])
    solver.Add(sum(inflow) - sum(outflow)  == 0)

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 42


### Define the objective
The following code defines the objective function for the example.

In [35]:

solver.Maximize(x[6,1])

### Call the solver
The following code calls the solver.

In [39]:
status = solver.Solve()

### Display the solution
The following code displays the solution.

In [38]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for i in Nodes:
        for j in Nodes:
            if x[i,j].solution_value() != 0:  # Just print the variables with non-zero value
                print(x[i,j].name(), ' = ', x[i,j].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 5.0
x[1,2]  =  2.0
x[1,3]  =  3.0
x[2,6]  =  2.0
x[3,4]  =  1.0
x[3,5]  =  2.0
x[4,6]  =  1.0
x[5,6]  =  2.0
x[6,1]  =  5.0

Problem solved in 333265.000000 milliseconds
Problem solved in 2 iterations
Problem solved in 1 branch-and-bound nodes


### Complete programs
Here are the complete programs.

In [41]:
solver = pywraplp.Solver.CreateSolver('SCIP')

def maxflowproblem():
    infinity = solver.infinity()
    x = {}
    for i in Nodes:
        for j in Nodes:
            x[i,j] = solver.IntVar(0, infinity, f'x[{i},{j}]')

    print('Number of variables =', solver.NumVariables())


    # Capacity of each arc:
    # You can also define this constraint as an upper bound of each decision variable in previous step
    for i in Nodes:
        for j in Nodes:
            solver.Add(x[i,j] <= Capacities[i-1][j-1]) # this -1 is because indexing in python starts from 0. 
                                                       # E.g. the capacity of node 1 to node 2 is Capacities[0,1]

    # flow balance at each node:
    for i in Nodes:
        inflow = []
        outflow = []
        for j in Nodes:
            inflow.append(x[j,i])
            outflow.append(x[i,j])
        solver.Add(sum(inflow) - sum(outflow)  == 0)

    print('Number of constraints =', solver.NumConstraints())


    # objective
    solver.Maximize(x[6,1])

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Objective value =', solver.Objective().Value())
        for i in Nodes:
            for j in Nodes:
                if x[i,j].solution_value() != 0:  # Just print the variables with non-zero value
                    print(x[i,j].name(), ' = ', x[i,j].solution_value())
        print()
        print('Problem solved in %f milliseconds' % solver.wall_time())
        print('Problem solved in %d iterations' % solver.iterations())
        print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    else:
        print('The problem does not have an optimal solution.')
        
maxflowproblem()

Number of variables = 36
Number of constraints = 42
Objective value = 5.0
x[1,2]  =  2.0
x[1,3]  =  3.0
x[2,6]  =  2.0
x[3,4]  =  1.0
x[3,5]  =  2.0
x[4,6]  =  1.0
x[5,6]  =  2.0
x[6,1]  =  5.0

Problem solved in 14.000000 milliseconds
Problem solved in 2 iterations
Problem solved in 1 branch-and-bound nodes


## Shortest Path Optimization Example


<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_97.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_98.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_99.webp?raw=1\" width="700">


### Declare the solver
In any MIP program, you start by importing the linear solver wrapper and declaring the MIP solver, as shown in the previous MIP example.

In [10]:
from ortools.linear_solver import pywraplp

### Create the data
The following code creates arrays containing the data for the example: the variable coefficients for the constraints and objective function, and bounds for the constraints.

In [58]:
Nodes = [i for i in range(1,7)]
m = 10e5  #we define a very large distance value for those nodes that dont have any link
Distance = [[m,100,200,m,m,m],
              [100,m,50,200,100,m],
              [200,50,m,m,40,m],
              [m,200,m,m,150,100],
              [m,100,40,150,m,100],
              [m,m,m,100,100,m]]

Balance = [-1,0,0,0,0,1]

### Instantiate the solver
The following code instantiates the solver.

In [54]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

### Define the variables
The following code defines the variables for the example in a loop. For large problems, this is easier than defining the variables individually, as in the previous example.

In [55]:
x = {}
for i in Nodes:
    for j in Nodes:
        x[i,j] = solver.BoolVar(f'x[{i},{j}]')  #use BoolVar for binary (boolean) decision variables. Also IntVar(0,1) can be used instead.

print('Number of variables =', solver.NumVariables())

Number of variables = 36


In [53]:
x

{(1, 1): x[1,1],
 (1, 2): x[1,2],
 (1, 3): x[1,3],
 (1, 4): x[1,4],
 (1, 5): x[1,5],
 (1, 6): x[1,6],
 (2, 1): x[2,1],
 (2, 2): x[2,2],
 (2, 3): x[2,3],
 (2, 4): x[2,4],
 (2, 5): x[2,5],
 (2, 6): x[2,6],
 (3, 1): x[3,1],
 (3, 2): x[3,2],
 (3, 3): x[3,3],
 (3, 4): x[3,4],
 (3, 5): x[3,5],
 (3, 6): x[3,6],
 (4, 1): x[4,1],
 (4, 2): x[4,2],
 (4, 3): x[4,3],
 (4, 4): x[4,4],
 (4, 5): x[4,5],
 (4, 6): x[4,6],
 (5, 1): x[5,1],
 (5, 2): x[5,2],
 (5, 3): x[5,3],
 (5, 4): x[5,4],
 (5, 5): x[5,5],
 (5, 6): x[5,6],
 (6, 1): x[6,1],
 (6, 2): x[6,2],
 (6, 3): x[6,3],
 (6, 4): x[6,4],
 (6, 5): x[6,5],
 (6, 6): x[6,6]}

### Define the constraints

The following code creates the constraints for the example, using loop.


In [59]:
# Capacity of each arc:
# You can also define this constraint as an upper bound of each decision variable in previous step

# flow balance at each node:
for i in Nodes:
    inflow = []
    outflow = []
    for j in Nodes:
        inflow.append(x[j,i])
        outflow.append(x[i,j])
    solver.Add(sum(inflow) - sum(outflow)  == Balance[i-1])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 6


### Define the objective
The following code defines the objective function for the example.

In [60]:
obj_expr = []
for i in Nodes:
    for j in Nodes:
        obj_expr.append(Distance[i-1][j-1] * x[i,j])
solver.Minimize(sum(obj_expr))

### Call the solver
The following code calls the solver.

In [61]:
status = solver.Solve()

### Display the solution
The following code displays the solution.

In [62]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for i in Nodes:
        for j in Nodes:
            if x[i,j].solution_value() != 0:  # Just print the variables with non-zero value
                print(x[i,j].name(), ' = ', x[i,j].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 290.0
x[1,2]  =  1.0
x[2,3]  =  1.0
x[3,5]  =  1.0
x[5,6]  =  1.0

Problem solved in 1015520.000000 milliseconds
Problem solved in 5 iterations
Problem solved in 1 branch-and-bound nodes


### Complete programs
Here are the complete programs.

In [64]:
from ortools.linear_solver import pywraplp

def shortestpath():
    Nodes = [i for i in range(1,7)]
    m = 10e5  #we define a very large distance value for those nodes that dont have any link
    Distance = [[m,100,200,m,m,m],
                  [100,m,50,200,100,m],
                  [200,50,m,m,40,m],
                  [m,200,m,m,150,100],
                  [m,100,40,150,m,100],
                  [m,m,m,100,100,m]]

    Balance = [-1,0,0,0,0,1]

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    x = {}
    for i in Nodes:
        for j in Nodes:
            x[i,j] = solver.BoolVar(f'x[{i},{j}]')  #use BoolVar for binary (boolean) decision variables. Also IntVar(0,1) can be used instead.

    print('Number of variables =', solver.NumVariables())

    # Capacity of each arc:
    # You can also define this constraint as an upper bound of each decision variable in previous step

    # flow balance at each node:
    for i in Nodes:
        inflow = []
        outflow = []
        for j in Nodes:
            inflow.append(x[j,i])
            outflow.append(x[i,j])
        solver.Add(sum(inflow) - sum(outflow)  == Balance[i-1])

    print('Number of constraints =', solver.NumConstraints())

    obj_expr = []
    for i in Nodes:
        for j in Nodes:
            obj_expr.append(Distance[i-1][j-1] * x[i,j])
    solver.Minimize(sum(obj_expr))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Objective value =', solver.Objective().Value())
        for i in Nodes:
            for j in Nodes:
                if x[i,j].solution_value() != 0:  # Just print the variables with non-zero value
                    print(x[i,j].name(), ' = ', x[i,j].solution_value())
        print()
        print('Problem solved in %f milliseconds' % solver.wall_time())
        print('Problem solved in %d iterations' % solver.iterations())
        print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    else:
        print('The problem does not have an optimal solution.')

shortestpath()

Number of variables = 36
Number of constraints = 6
Objective value = 290.0
x[1,2]  =  1.0
x[2,3]  =  1.0
x[3,5]  =  1.0
x[5,6]  =  1.0

Problem solved in 7.000000 milliseconds
Problem solved in 5 iterations
Problem solved in 1 branch-and-bound nodes
